In [ ]:

import os

In [ ]:
os.chdir('..')
os.getcwd()

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    image_size: list
    learning_rate: float
    AUGMENTATION: bool
    momentum: float
    optimizer: str
    classes: int


In [ ]:
from src.collision_alert.constants import *
from src.collision_alert.utils import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            image_size=self.params.IMAGE_SIZE,
            learning_rate=self.params.LEARNING_RATE,
            AUGMENTATION=self.params.AUGMENTATION,
            momentum=self.params.momentum,
            optimizer = self.params.optimizer,
            classes=self.params.CLASSES
        )

        return prepare_base_model_config
        

In [ ]:
#!pip install ultralytics -q
from ultralytics import YOLO
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        print(self.config.base_model_path)

    def get_base_model(self):
        self.model = YOLO('yolov8m.yaml')  # build a new model from YAML
        self.model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)
        self.model = YOLO('yolov8m.yaml').load('yolov8m.pt')  # build from YAML and transfer weights

        self.model.save(Path(self.config.base_model_path))

        self.save_model(path=self.config.base_model_path, model=self.model)
    @staticmethod
    def save_model(path: Path, model):
        model.save(path)

In [ ]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e
